As a part of this notebook we are trying to extend the dataset by downloading the content and title of the article.
the extended dataset is saved with four additional columns:
- title
- content
- label_by_mean
- label_by_median

> 0 represents will not go viral.
> 1 represents will go viral

In [1]:
! pip install tqdm

In [2]:
import pandas as pd
import numpy as np
from pprint import pprint as pp
from tqdm import tqdm
import string
from newspaper import Article, ArticleException
table = str.maketrans("","",string.punctuation)

In [3]:
uci = pd.read_csv("../uci_data/OnlineNewsPopularity.csv", delimiter=", ")
pp(len(uci))

uci['title'] = ""
uci['content'] = ""
uci['label_by_mean'] = 0
uci['label_by_median'] = 0
pp(uci.columns)

e:\project_viralnews\venv\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


39644
Index(['url', 'timedelta', 'n_tokens_title', 'n_tokens_content',
       'n_unique_tokens', 'n_non_stop_words', 'n_non_stop_unique_tokens',
       'num_hrefs', 'num_self_hrefs', 'num_imgs', 'num_videos',
       'average_token_length', 'num_keywords', 'data_channel_is_lifestyle',
       'data_channel_is_entertainment', 'data_channel_is_bus',
       'data_channel_is_socmed', 'data_channel_is_tech',
       'data_channel_is_world', 'kw_min_min', 'kw_max_min', 'kw_avg_min',
       'kw_min_max', 'kw_max_max', 'kw_avg_max', 'kw_min_avg', 'kw_max_avg',
       'kw_avg_avg', 'self_reference_min_shares', 'self_reference_max_shares',
       'self_reference_avg_sharess', 'weekday_is_monday', 'weekday_is_tuesday',
       'weekday_is_wednesday', 'weekday_is_thursday', 'weekday_is_friday',
       'weekday_is_saturday', 'weekday_is_sunday', 'is_weekend', 'LDA_00',
       'LDA_01', 'LDA_02', 'LDA_03', 'LDA_04', 'global_subjectivity',
       'global_sentiment_polarity', 'global_rate_positive_words',

In [4]:
mean = uci.shares.mean()
median = uci.shares.median()
maxi = uci.shares.max()
mini = uci.shares.min()
mean, median, maxi, mini

(3395.3801836343455, 1400.0, 843300, 1)

In [5]:
uci.loc[uci.shares < mean,'shares'].count()/len(uci), uci.loc[uci.shares < median,'shares'].count()/len(uci)

(0.7962112803955201, 0.46640096862072444)

In [6]:
uci.loc[uci.shares<mean,'label_by_mean'] = 1
uci.loc[uci.shares<median,'label_by_median'] = 1

In [7]:
not_parsed = set()
for idx, row in tqdm(uci.iterrows(), total=len(uci)):
#     row = uci.loc[idx]  
    article = Article(row.url)
    article.download()
    try:
        article.parse()
        uci.loc[uci.url == row.url, 'title']=article.title.translate(table)
        uci.loc[uci.url == row.url, 'content']=[article.text.translate(table)]
        if idx%100 == 0 and idx != 0:
            uci[:idx].to_csv("../uci_data/OnlineNewsPopularity_extended_{}.csv".format(idx))
#             break
    except ArticleException as e:
        pass
        not_parsed.add(row['url'])
#         print(str(e))
#         print(row['url'])
uci.to_csv("../uci_data/OnlineNewsPopularity_extended.csv")        
print(not_parsed)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39644/39644 [7:32:15<00:00,  1.46it/s]


{'http://mashable.com/2014/02/19/just-like-dad/', 'http://mashable.com/2014/07/02/how-delete-facebook/', 'http://mashable.com/2013/10/16/young-adult-books-cry/', 'http://mashable.com/2014/05/19/nutella-recipe/', 'http://mashable.com/2014/01/24/mac-name-facts/', 'http://mashable.com/2014/02/07/the-beatles-50-years/', 'http://mashable.com/2014/05/07/creative-family-portraits/', 'http://mashable.com/2014/03/05/sun-stars-space-facts/', 'http://mashable.com/2014/12/16/los-angeles-cosby-no-charges/', 'http://mashable.com/2014/12/22/best-content-delivery-networks/', 'http://mashable.com/2014/06/06/son-surprises-father-with-car/', 'http://mashable.com/2014/06/06/d-day-code-names/', 'http://mashable.com/2013/03/09/prizm-home/', 'http://mashable.com/2014/06/10/mobile-apps-organize/', 'http://mashable.com/2013/11/11/family-resources/', 'http://mashable.com/2013/12/03/build-mobile-apps/', 'http://mashable.com/2014/03/25/james-rebhorn-obituary/', 'http://mashable.com/2014/04/21/how-to-sign/', 'http

In [8]:
uci[['title','content']].head()

,title,content
0,Amazons Streaming Video Library Now a Little E...,Having trouble finding something to watch on A...
1,APs Twitter to Begin Displaying Sponsored Tweets,The Associated Press is the latest news organi...
2,Apples App Store Passes 40 Billion Downloads,It looks like 2012 was a pretty good year for ...
3,This Astronaut Is Rooting for Notre Dame Tonight,When it comes to college football NASA astrona...
4,New UVerse Apps Simplify Sharing Photos and Vi...,LAS VEGAS — Sharing photos and videos on your ...


In [9]:
uci[['title','content']].tail()

,title,content
39639,Samsung app aims to improve autistic childrens...,While some believe smartphones and tablets may...
39640,Seth Rogen James Franco are planning to livetw...,LOS ANGELES — Call it their exit Interview\n\n...
39641,App developer says Merry Christmas by paying o...,Nothing says Merry Christmas like never having...
39642,Two dead after Ukraine rocked by series of blasts,Ukrainians were on high alert on Saturday afte...
39643,8 YouTube channels to watch in 2015,We collectively watch more than 6 billion hour...


In [10]:
sentence = "This sentence has, loads/ of punc-tuations??!"
sentence.translate(table), string.punctuation

('This sentence has loads of punctuations',
 '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~')